In [1]:
import numpy as np

import plotly.express as px
import plotly.io as pio
pio.templates.default = 'plotly_dark'

In [2]:
import tensorflow as tf

## NN из одного нейрона для умножения числа на 3

In [3]:
from keras.models import Sequential # sequense of layers in NN

from keras.layers import Dense # класс полносвязного/линейного слоя
from keras.layers import Input # класс входного слоя

from keras.utils import set_random_seed
set_random_seed(42)

In [ ]:

# Создадим модель с одним слоем (он же будет выходным слоем),
model = Sequential([
    Input((1,)),
    Dense(
        1, # в котором будет всего один нейрон.
        activation='linear' # Используем линейную активацию
    )
])
model.summary()

w, b = model.get_weights()
print("Веса модели инициализированы случайными значениями:")
print(f"w = {w[0,0]}, b = {b[0]}")

In [27]:
# "Скомпилируем" модель, то есть зададим:
model.compile(
    optimizer='sgd', # оптимизатор (стохастический град спуск),
    loss='mse',      # функцию потерь, которая будет минимизироваться,
    metrics=['mae']  # метрики
)

In [28]:
X_train = np.array([[1], [2], [3], [4], [5], [6], [7], [8], [9], [10]])
y_train = np.array([3, 6, 9, 12, 15, 18, 21, 24, 27, 30])

In [ ]:
model.fit(X_train, y_train, epochs=100)

In [ ]:
model.get_weights()

In [ ]:
model.predict(np.array([[-5], [33]]))

## NN для сложения двух чисел

In [32]:
# Сгенерируем обучающие данные
X1 = np.random.randint(1, 10, size=100).reshape(-1,1)
X2 = np.random.randint(1, 10, size=100).reshape(-1,1)
X_train = np.hstack([X1, X2])
y_train = X1 + X2

Поскольку НС использует для оптимизации градиентный спуск входные данные нужно отмасштабировать:

In [33]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X_train)

Создадим НС с двумя входыми и двумя слоями: скрытый слой и выходной слой

<img src="images/NN2.jpeg" width=500px>

In [ ]:
model = Sequential([
    Input((2,)),
    Dense(3, activation='linear'),
    Dense(1, activation='linear')
])
model.summary()
model.get_weights()

In [ ]:
model.compile(optimizer='sgd', loss='mse', metrics=['mape'])

model.fit(X_scaled, y_train, epochs=100)

In [ ]:
model.predict(scaler.transform(np.array([[-10, 33]])))

## NN для датасета *MNIST*

Картинки с рукописными цифрами 28 $\times$ 28 пикселей

In [4]:
from keras.datasets import mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train.shape, X_test.shape

((60000, 28, 28), (10000, 28, 28))

In [5]:
px.imshow(np.hstack(X_train[:5]), width=1000, height=300).update_coloraxes(showscale=False).show()
y_train[:5]

array([5, 0, 4, 1, 9], dtype=uint8)

Уменьшим размер картинок для сокращения количества признаков:

In [6]:
reduced_size = (14, 14)

X_train_resized = tf.image.resize(X_train[..., np.newaxis], reduced_size)[..., 0]
X_test_resized = tf.image.resize(X_test[..., np.newaxis], reduced_size)[..., 0]

px.imshow(np.hstack(X_train_resized[:5]), width=1000, height=300).update_coloraxes(showscale=False).show()

In [7]:
print(f"Диапазон значений пикселей: от {X_train.min()} до {X_train.max()}")

Диапазон значений пикселей: от 0 до 255


Нормируем данные простым делением на максимальное значение (255):

In [8]:
X_train_scaled = X_train_resized / 255
X_test_scaled = X_test_resized / 255

In [9]:
from keras.utils import to_categorical

y_train_cat = to_categorical(y_train)
y_train_cat.shape

(60000, 10)

In [10]:
from keras.layers import Flatten # класс входного слоя с переводом в одномерный массив

model = Sequential([
    Flatten(),
    Dense(10, activation='sigmoid')
])

model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])

In [11]:
model.fit(X_train_scaled, y_train_cat, epochs=50)

Epoch 1/50
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 1s 425us/step - accuracy: 0.5551 - loss: 1.6840
Epoch 2/50
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 1s 441us/step - accuracy: 0.8381 - loss: 0.7975
Epoch 3/50
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 1s 440us/step - accuracy: 0.8608 - loss: 0.6269
Epoch 4/50
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 1s 442us/step - accuracy: 0.8699 - loss: 0.5509
Epoch 5/50
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 1s 446us/step - accuracy: 0.8764 - loss: 0.5064
Epoch 6/50
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 1s 425us/step - accuracy: 0.8809 - loss: 0.4766
Epoch 7/50
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 1s 403us/step - accuracy: 0.8843 - loss: 0.4550
Epoch 8/50
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 1s 395us/step - accuracy: 0.8865 - loss: 0.4384
Epoch 9/50
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 1s 409us/step - accuracy: 0.8886 - loss: 0.4253
Epoch 10/50
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 1s 404us/step - accuracy: 0.8902 - loss: 0.4145
Epoch 11/50
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 1s 397us/step - accuracy: 0.8920 - loss: 0.4054
Epoch 12

In [12]:
y_test_pred = model.predict(X_test_scaled).argmax(axis=1)

from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_test_pred)

313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 287us/step


0.9149